<a href="https://colab.research.google.com/github/p-owens/MAI/blob/main/shap_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install shap

In [ ]:
#allow file access
from google.colab import files

from tensorflow.keras.models import load_model

import pickle

#import shap  
import shap
shap.initjs()

import pandas as pd
import tensorflow as tf
tf.compat.v1.disable_v2_behavior() #this was required for SHAP values
from tensorflow import keras


#borrowed form 5c16
from keras import datasets
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.regularizers import l2

from keras.models import model_from_json

from IPython.display import clear_output

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MaxNLocator

import pickle
import sklearn as skl

from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_score

In [ ]:
ch_names = np.arange(44).tolist()
freqs = np.arange(191.6, 195.9, 0.1).tolist()
for x in range(len(ch_names)):
  ch_names[x] = "Ch %.0f (%.1f)" % (ch_names[x], freqs[x])

In [ ]:
#retreiving both datasets from GitHub
git_xval = 'https://raw.githubusercontent.com/p-owens/MAI/main/x_val.csv'
git_yval = 'https://raw.githubusercontent.com/p-owens/MAI/main/y_val.csv'
df0 = pd.read_csv(git_xval,
                  index_col=False,
                  header=None,
                  names=ch_names                                   
                 )
df1 = pd.read_csv(git_yval,
                  index_col=False,
                  header=None,
                  names=ch_names
                 )

x_values = df0.to_numpy()
y_values = df1.to_numpy()

#removing the frequency row - treating these values as labels
x_values = x_values[1::2,:]

#generate the validation and training datasets
x_train, x_val, y_train, y_val = skl.model_selection.train_test_split(x_values, y_values, test_size=.2, random_state=0)#renaimed x_rem and y_rem to x_values and y_values

In [ ]:
#set the channel number here
channel = 35

In [ ]:
#Using a dataset with 1 ch constantly on for the test set
#retreiving both datasets from GitHub



git_xval = ('https://raw.githubusercontent.com/p-owens/MAI/main/data%20gen/ch{0}/x_val.csv'.format(channel))
git_yval = ('https://raw.githubusercontent.com/p-owens/MAI/main/data%20gen/ch{0}/y_val.csv'.format(channel))
dfx = pd.read_csv(git_xval,
                  index_col=False,
                  header=None,
                  names=ch_names                                   
                 )
dfy = pd.read_csv(git_yval,
                  index_col=False,
                  header=None,
                  names=ch_names
                 )
x_test = dfx.to_numpy()
y_test = dfy.to_numpy()

#removing the frequency row - treating these values as labels
x_test = x_test[1::2,:]

In [ ]:
#Load a previously saved model
model = tf.keras.models.load_model('model_entire_training_set_l_0.13_me_0.18')
model.summary()

In [ ]:
#Generate the SHAP values for the model
background = x_train[np.random.choice(x_train.shape[0], 4999, replace=False)]
explain = shap.DeepExplainer(model, background)

In [ ]:
#~45 mins to evaluate, ~2300 vals
#~15 mins to evaluate, 500 vals
shap_values = explain.shap_values(x_test)

In [ ]:
#saving the shap model evaluated on the test set
filename = 'shap_vals_ch{0}.txt'.format(channel)

with open(('{0}'.format(filename)), 'wb') as fv:
   pickle.dump(shap_values, fv)

files.download(('{0}'.format(filename)))

In [ ]:
#computing the shap values for the effect that all the other channels have on channel 40 (in this case) 
sv_cpy = shap_values[channel].copy()
sv_cpy[:,channel] = 0
avg_vec = np.average(np.absolute(sv_cpy), 0)
all_avg_s[channel, 1:] = avg_vec

#saving the matrix as a .csv
df_save = pd.DataFrame(all_avg_s, columns=(['Weights'] + ch_names))
df_save.to_csv('all_mean_shap.csv', header=(['Weights'] + ch_names))
files.download('all_mean_shap.csv')

In [ ]:
ch = channel
print(explain.expected_value[ch])
shap.summary_plot(sv_cpy, x_test, sort=True, feature_names=ch_names, plot_type="bar")

In [ ]:
ch = channel
print(explain.expected_value[ch])
shap.summary_plot(shap_values[ch], x_test, sort=True, feature_names=ch_names , plot_type="bar")